# Text Similarity with GPT-2 (version 1)

## 1. Fine-tuning GPT-2 for Text Similarity

In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the readerbench dataset
dataset = load_dataset("readerbench/ro_fake_news", "eda")

considered_tag_words = ['5g', 'vaccinare', 'controlul']


# Use only "headline" and "body" for the text and "tags" for similarity labels
def preprocess_data(example):
    # Combine the headline and body to form the input text
    example['text'] = example['headline'] + " " + example['body']

    # Check the 'tags' field
    tags = example['tags']
    # print(f"Tags Field: {tags}")  # Debugging the tags field

    # Check if any of the considered tag words appear in example['tags']
    found = any(word in tags.lower() for word in considered_tag_words) if tags else False
    example['label'] = 1.0 if found else 0.0

    # Debugging print statements
    # print(f"Processed Example -> Tags: {tags} | Found: {found} | Label: {example['label']}")

    return example


# Apply preprocessing to each subset in the dataset
dataset = dataset.map(preprocess_data)

# Concatenate all subsets (train, validation, test) into a single dataset
combined_dataset = concatenate_datasets([dataset["train"], dataset["validation"], dataset["test"]])

# Split the combined dataset into a new train and validation set
dataset_split = combined_dataset.train_test_split(test_size=0.1)
train_data = dataset_split["train"]
eval_data = dataset_split["test"]

# Load the GPT-2 tokenizer and model for sequence classification
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=1)

# Add a pad token to the tokenizer and model
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Move the model to the GPU
model.to(device)

# Tokenize the data
def tokenize_data(example):
    encoding = tokenizer(
        example['text'],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    encoding['labels'] = example['label']
    return encoding


# Apply tokenization to train and validation sets
train_data = train_data.map(tokenize_data)
eval_data = eval_data.map(tokenize_data)

train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir="../gpt2-similarity",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./gpt2-similarity")
tokenizer.save_pretrained("./gpt2-similarity")


# Testing the fine-tuned model with a custom prompt
def calculate_similarity(prompt, ground_truth_text):
    # Tokenize inputs and move them to the GPU if available
    inputs = tokenizer(prompt + " " + ground_truth_text, return_tensors="pt", truncation=True, padding="max_length",
                       max_length=512).to(device)

    # Run inference
    with torch.no_grad():
        outputs = model(**inputs)
        similarity_score = torch.sigmoid(outputs.logits).item()  # Convert logits to probability
    return similarity_score


# Example usage
prompt = "This is a new article about Romanian politics."
ground_truth_text = "Some fake news headline and body text."
similarity = calculate_similarity(prompt, ground_truth_text)
print(f"Similarity score between prompt and ground truth: {similarity:.2f}")


C:\global_envs\pytorch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 59/59 [00:00<00:00, 168.65 examples/s]
C:\global_envs\pytorch_env\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.186267
2,No log,0.159469
3,No log,0.145629


Similarity score between prompt and ground truth: 0.98


## 2. Loading and Testing the Fine-Tuned Model

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("../gpt2-similarity")
model = GPT2ForSequenceClassification.from_pretrained("../gpt2-similarity")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a function to calculate similarity
def calculate_similarity(prompt, ground_truth_text):
    inputs = tokenizer(prompt + " " + ground_truth_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(**inputs)
        similarity_score = torch.sigmoid(outputs.logits).item()  # Convert logits to probability
    return similarity_score

def test_similarity(prompt, ground_truth_text):
    similarity = calculate_similarity(prompt, ground_truth_text)
    print(f"Similarity score between prompt and ground truth: {similarity:.2f}")

# Test the model with a custom prompt and ground truth
prompt1 = "In Regatul Unit se răspândește un val de teorii ale conspirației despre COVID-19 si 5G."
prompt2 = "Regatul Unit, la fel ca multe alte state, se confruntă cu o creștere a numărului de teorii ale conspirației. Conspirații care variază de la ideea că virusul COVID-19 nu există la teoria că a fost creat în laborator de către chinezi pentru un război biologic, până la teoria că există o legătură între rețelele de tip 5G și virus, împreună cu multe altele, au fost propagate cu ușurință în spațiul virtual, majoritatea prin intermediul platformelor de social media"
prompt3 = "In Regatul Unit nu se răspândește niciun val de teorii ale conspirației despre COVID-19 si 5G."
ground_truth_text = "Regatul Unit, la fel ca multe alte state, se confruntă cu o creștere a numărului de teorii ale conspirației. Conspirații care variază de la ideea că virusul COVID-19 nu există la teoria că a fost creat în laborator de către chinezi pentru un război biologic, până la teoria că există o legătură între rețelele de tip 5G și virus, împreună cu multe altele, au fost propagate cu ușurință în spațiul virtual, majoritatea prin intermediul platformelor de social media"

test_similarity(prompt1, ground_truth_text)
test_similarity(prompt2, ground_truth_text)
test_similarity(prompt3, ground_truth_text)

Similarity score between prompt and ground truth: 0.60
Similarity score between prompt and ground truth: 0.52
Similarity score between prompt and ground truth: 0.60


## 3. Checking for GPU Availability

In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if a GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.current_device())  # Index of the current GPU

True
1
0
